# Setup

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
import yaml
import os

In [2]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [3]:
openai = ChatOpenAI(model_name='gpt-4.1-nano', temperature=0)

# SequentialChain

In [4]:
# Carregar os documentos
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()
# Carregar histórico de conversas 
historico_conversas = """Cliente: Minha britadeira não liga. Chatbot: Você já verificou 
                         se a bateria está carregada e conectada corretamente?"""
# Pergunta do cliente
pergunta = "Minha britadeira não liga. Eu já veriquei e a bateria está carregada e conectada corretamente"

In [5]:
inputs = {
    "context": "\n".join(doc.page_content for doc in documents),
    "question": pergunta,
    "historico": historico_conversas
}

In [6]:
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use o seguinte contexto para responder à pergunta. 
    Responda apenas com base nas informações fornecidas.
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Contexto: {context}
    Pergunta: {question}"""
)
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Use o histórico de conversas para responder à pergunta. 
    Responda apenas com base nas informações fornecidas. 
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Histórico: {historico}
    Pergunta: {question}"""
)
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as seguintes respostas para gerar uma resposta final,
    mas não forneça instruções de procedimentos já realizados:
    Resposta da base de conhecimento: {resposta_base_conhecimento}
    Resposta do histórico de conversas: {resposta_historico_conversas}"""
)

In [7]:
print(prompt_base_conhecimento)

input_variables=['context', 'question'] template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}'


In [8]:
# Definir as cadeias  
chain_base_conhecimento = prompt_base_conhecimento | openai
chain_historico_conversas = prompt_historico_conversas | openai
chain_final = prompt_final | openai

In [ ]:
print(chain_base_conhecimento)

In [ ]:
print(chain_historico_conversas)

In [9]:
# Passando dados e executando
resultado_base_conhecimento = chain_base_conhecimento.invoke({"context": inputs["context"], "question": inputs["question"]})
resultado_historico_conversas = chain_historico_conversas.invoke({"historico": inputs["historico"], "question": inputs["question"]})
resultado_final = chain_final.invoke({"resposta_base_conhecimento": resultado_base_conhecimento, 
                                      "resposta_historico_conversas": resultado_historico_conversas})

In [10]:
print("Resultado Base de Conhecimento:\n", resultado_base_conhecimento)
print("----")
print("Resultado Histórico de Conversas:\n", resultado_historico_conversas)

Resultado Base de Conhecimento:
 content='Verifique se o interruptor liga/desliga está na posição correta. Se o problema persistir, aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Se ainda assim não ligar, contate o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 681, 'total_tokens': 738, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'finish_reason': 'stop', 'logprobs': None} id='run-0a7631de-eb67-4cc1-8630-28bfd3b91e28-0' usage_metadata={'input_tokens': 681, 'output_tokens': 57, 'total_tokens': 738}
----
Resultado Histórico de Conversas:
 content='Se a bateria está carregada e conectada corretamente, pode haver outro problema. Recomendo verificar se há algum

In [11]:
print(resultado_final.content)

Se a bateria está carregada e conectada corretamente, verifique se o interruptor liga/desliga está na posição correta. Caso o problema persista, aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Se mesmo assim não ligar, pode haver outro problema no motor ou no sistema elétrico da britadeira. Nesse caso, recomenda-se contatar o suporte técnico pelo 0800 555 5555.


In [12]:
print(resultado_final)

content='Se a bateria está carregada e conectada corretamente, verifique se o interruptor liga/desliga está na posição correta. Caso o problema persista, aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Se mesmo assim não ligar, pode haver outro problema no motor ou no sistema elétrico da britadeira. Nesse caso, recomenda-se contatar o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 511, 'total_tokens': 599, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'finish_reason': 'stop', 'logprobs': None} id='run-6fc4097c-f809-478b-8948-7f8403618ab1-0' usage_metadata={'input_tokens': 511, 'output_tokens': 88, 'total_tokens': 599}
